<a href="https://colab.research.google.com/github/tourbut/MedicalAI/blob/main/mpt_7b_instruction_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests torch transformers einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.3 MB/s eta 0:00:00


# Mosaic Instruct Model MPT-7B-Instruct



In [ ]:
from typing import Any, Dict, Tuple
import warnings
import datetime
import os
from threading import Event, Thread
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

import textwrap

INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)


class InstructionTextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.5,
            "top_p": 0.92,
            "top_k": 0,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.1,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def format_instruction(self, instruction):
        return PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        s = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction)
        input_ids = self.tokenizer(s, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # Slice the output_ids tensor to get only new tokens
        new_tokens = output_ids[0, len(input_ids[0]) :]
        output_text = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
        return output_text


In [ ]:
generate = InstructionTextGenerationPipeline(
    "mosaicml/mpt-7b-instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/bd1748ec173f1c43e11f1973fc6e61cb3de0f327/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-3-0cb7683e5811>:48: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [ ]:
!nvidia-smi

Tue May  9 04:23:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W /  70W |  13341MiB / 15360MiB |     32%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def process_stream(instruction, temperature, top_p, top_k, max_new_tokens):
    # Tokenize the input
    input_ids = generate.tokenizer(
        generate.format_instruction(instruction), return_tensors="pt"
    ).input_ids
    input_ids = input_ids.to(generate.model.device)

    # Initialize the streamer and stopping criteria
    streamer = TextIteratorStreamer(
        generate.tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )
    stop = StopOnTokens()

    if temperature < 0.1:
        temperature = 0.0
        do_sample = False
    else:
        do_sample = True

    gkw = {
        **generate.generate_kwargs,
        **{
            "input_ids": input_ids,
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": do_sample,
            "top_p": top_p,
            "top_k": top_k,
            "streamer": streamer,
            "stopping_criteria": StoppingCriteriaList([stop]),
        },
    }

    response = ''
    

    def generate_and_signal_complete():
        generate.model.generate(**gkw)

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    for new_text in streamer:
        response += new_text
   
    return response

# Try Your Prompts

In [ ]:
instruction = "Write a travel blog about a 3-day trip to The Philippines. You need describe day by day."
temperature = 0.3
top_p = 0.95
top_k = 0
max_new_tokens = 2000
response = process_stream(instruction, temperature, top_p, top_k, max_new_tokens)

wrapped_text = textwrap.fill(response, width=100)
print(wrapped_text +'\n\n')

Day 1 - Arrived in Manila, met up with my friend and went out for dinner at this restaurant called
Cafe Adriatico which was recommended on Trip Advisor. It's located inside a hotel but you don't have
to stay there if you're not staying as it has its own entrance from outside. I had their signature
dish of Pork Belly and it came with 2 sides (I chose mashed potatoes) plus rice. My friends ordered
different dishes like Chicken Parmigiana and Beef Stroganoff. We also shared some desserts between
us such as Chocolate Fondant Cake and Strawberry Cheesecake. After our meal we walked around
Intramuros area where most of the historical buildings are found before heading back to our Airbnb
place nearby. Day 2 - Started off early morning going to Baclaran market near Pasay City then took a
cab to Tagaytay city via Star Tollway. Had lunch at one of the restaurants along the way named "The
Bistro" which served good food and great view overlooking Taal Lake. Then continued driving further
south tow

In [ ]:
import markdown
instruction = "Write Python code to scrape the listing from eBay.com"
temperature = 0.1
top_p = 0.95
top_k = 0
max_new_tokens = 1000
response = process_stream(instruction, temperature, top_p, top_k, max_new_tokens)

wrapped_text = textwrap.fill(response, width=100)
print(markdown.markdown(wrapped_text +'\n\n'))

<p>from bs4 import BeautifulSoup as Soup  import requests   url =
'https://www.ebay.ca/sch/i.html?_nkw=iphone+12&amp;_sop=1' #change this URL with your own search query
response = requests.get(url)   #make HTTP GET Request on given url     page_source = response.text</p>
<h1>save page source in variable called "page_source"       soup = Soup(page_source, 'html.parser')</h1>
<h1>parse HTML using beautiful soup library and save it into another variable named "soup".         for</h1>
<p>item in soup.findAll('div', {'class':'item-title'}):          #search for all div tags which has
class attribute equal to "item-title", then iterate over them one by one                    print
(item.a['href'])</p>


In [ ]:
instruction = "Solve the equation for x: \'log_2(x) + log_2(x - 4) = 3\'"

temperature = 0.5
top_p = 0.95
top_k = 0
max_new_tokens = 1000
response = process_stream(instruction, temperature, top_p, top_k, max_new_tokens)

wrapped_text = textwrap.fill(response, width=100)
print(wrapped_text +'\n\n')

The solution to this problem can be found by using logarithms and addition properties of logs,
specifically: "the sum of two logarithms equals the natural logarithm of their product."  This
yields;  "ln(x) + ln((x-4)) = ln(3)" which simplifies into:     "x = 2^(ln(3)/ln(2))"


